In [1]:
import chromadb
import asyncio
import ollama
import os

In [2]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

# collection = chroma_client.get_or_create_collection(name='docs')

In [3]:
collection = chroma_client.get_or_create_collection(name='docs')

In [4]:
# 預先將會重複使用的功能，以函數的方式，便於後續呼叫使用

In [5]:
def heartbeat():
    return chroma_client.heartbeat()

def delete_collection(collection):
    chroma_client.delete_collection(collection)

def collection_reset():
    data_ids = collection.peek(100)['ids']
    for i in data_ids:
        # print(i)
        collection.delete(i)

def collection_add(i, embedding, document):
    collection.add(
        ids=[i],
        embeddings=[embedding],
        documents=[document]
    )

def collection_add_v2(i, embedding, document, metadata):
    collection.add(
        ids=[i],
        embeddings=[embedding],
        documents=[document],
        metadatas=[metadata]
    )

def collection_query(query, n_results):
    results = collection.query(
        query_embeddings=query,
        n_results=n_results
    )
    return results

In [24]:
# 清除 docs 這個 collection 內所有 data
while True:
    data_ids = collection.peek()['ids']
    if not data_ids:
        break
    for i in data_ids:
        # print(i)
        collection.delete(i)

In [7]:
##### Demo of usage abt collection.metadatas #####
# import chromadb

# client = chromadb.PersistentClient(path="my_local_data")  # or HttpClient()

# collection = client.get_or_create_collection("local_collection")
# collection.add(
#     ids=[f"{i}" for i in range(1000)],
#     documents=[f"document {i}" for i in range(1000)],
#     metadatas=[{"doc_id": i} for i in range(1000)])
# existing_count = collection.count()
# batch_size = 10
# for i in range(0, existing_count, batch_size):
#     batch = collection.get(
#         include=["metadatas", "documents", "embeddings"],
#         limit=batch_size,
#         offset=i)
#     print(batch)  # do something with the batch

path2dataset = './dataset/'
def read_documents_from_directory(directory):
    documents = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                documents.append(file.read())
                filenames.append(filename)
    return documents, filenames

documents, filenames = read_documents_from_directory(path2dataset)

# print(f"document: {documents}")
print(f"filenames: {filenames}")

filenames: ['EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'EP228___海底抓取機器人_研發與實作的進程_博士的心境培養_ft__子源.txt', 'EP226___雲原生系統設計_後端微服務開發_DevOps_是種文化_ft__Tony.txt', 'EP223___中台資工系差異_學生接案經驗談_創新與洞察需求_ft__Yuan_Lin.txt', 'EP225___哥倫比亞研究生_紐約生活與衝擊_創新開發的學習_ft__Tony.txt', 'EP230___工程師紓壓策略_心理健康的警訊_建立興趣與支持網_ft__Peter.txt', 'EP221___微軟工程師分享_大外商運作文化_AI_提升生產力_ft__藍恩.txt', 'EP224___畢業生立即創業_雲端與_PaaS_開發_學生創新與紅利__ft__Yuan_Lin.txt', 'EP220___工程師媒體經營_人脈建立的橋樑_利他與善的循環_ft__this_web_請網這邊走.txt', 'EP222___工程師_X_站立喜劇_理工仔的笑話思維_表演與工作的平衡_ft__藍恩.txt', 'EP219___機電系轉前端_自學網頁歷程_做中學的分享_ft__this_web_請網這邊走.txt', 'EP229___工程師職場壓力_精神健康的覺察_生活韌性的提升_ft__Peter.txt']


In [9]:
token_size = 512
shift_size = 256

In [ ]:

for i, (f, d) in enumerate(zip(filenames, documents)):
    print(f)
    # print(d)
    response = ollama.embeddings(model="llama3", prompt=d)
    embedding = response["embedding"]
    # print(embedding)
    metadata = {"type":"podcast", "name":"techporn", "ids":f}
    print(metadata)
    collection_add_v2(str(i), embedding, d, metadata)

In [13]:
def process_document(doc, model="llama3"):
    length = len(doc)
    start = 0
    embeddings = []
    chunks = []

    while start < length:
        end = min(start + token_size, length)
        chunk = doc[start:end]
        response = ollama.embeddings(model=model, prompt=chunk)
        embeddings.append(response["embedding"])
        chunks.append(chunk)
        start += shift_size  # Move start forward by the shift size
        

    # return chunks
    return embeddings, chunks

for i, (f, d) in enumerate(zip(filenames, documents)):
    print(f)
    embeddings, chunks = process_document(d)
    # chunks = process_document(d)
    for idx, (embedding, chunk) in enumerate(zip(embeddings, chunks)):
    # for idx, chunk in enumerate(chunks):
        metadata = {"type": "podcast", "name": "techporn", "episode": f, "id": f"{idx}"}
        print(metadata)
        print(f"{i}-{idx}")
        collection_add_v2(f"{i}-{idx}", embedding, chunk, metadata)

EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-0'}
0-0
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-1'}
0-1
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-2'}
0-2
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-3'}
0-3
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-4'}
0-4
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-5'}
0-5
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-6'}
0-6
{'type': 'podcast', 'name': 'techporn', 'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-7'}
0-7
{'type': 'podcast', 'name': 'techporn', 'epis

In [22]:
documents = [
    "動物界,脊索動物門,哺乳綱,嚙齒目,鼠科,大鼠屬,溝鼠",
    "動物界,脊索動物門,哺乳綱,嚙齒目,倉鼠科,田鼠屬,田鼠",
    "動物界,脊索動物門,哺乳綱,嚙齒目,松鼠科,麗松鼠屬,赤腹鼠",
    "動物界,脊索動物門,哺乳綱,雙門齒目,袋鼠科,袋鼠屬,紅大袋鼠",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,牛屬,黃牛",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,水牛屬,水牛",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,孟加拉虎",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,獅子",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,豹屬,花豹",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,石虎屬,石虎",
    "動物界,脊索動物門,哺乳綱,肉食目,貓科,獵豹屬,獵豹",
    "動物界,脊索動物門,哺乳綱,兔形目,兔科,穴兔屬,家兔",
    "動物界,脊索動物門,爬蟲綱,有鱗目,眼鏡蛇科,環蛇屬,雨傘節",
    "動物界,脊索動物門,爬蟲綱,有鱗目,蝰科,青竹絲屬,青竹絲",
    "動物界,脊索動物門,哺乳綱,奇蹄目,馬科,馬屬,馬",
    "動物界,脊索動物門,哺乳綱,奇蹄目,馬科,馬屬,斑馬",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,山羊屬,山羊",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,盤羊屬,綿羊",
    "動物界,脊索動物門,哺乳綱,偶蹄目,牛科,鬣羚屬,台灣長鬃山羊",
    "動物界,脊索動物門,哺乳綱,靈長目,猴科,獼猴屬,台灣獼猴",
]
# 檢查每個文件 ID 是否已存在，如果存在則跳過新增操作
existing_docs = collection.get()
existing_ids = set(existing_docs['ids'])

for i, d in enumerate(documents):
    response = ollama.embeddings(model="llama3", prompt=d)
    embedding = response["embedding"]
    collection_add(str(i),embedding,d)
    # print(f"embeddings: [{embedding}], documents: [{d}]")
    # collection.add(
    #   ids=[str(i)],
    #   embeddings=[embedding],
    #   documents=[d]
    # )

print('done')


done


In [35]:
prompt = "stand up comedy"
extend_prompts = ollama.generate(
    model="llama3",
    prompt=f"請用繁體中文，以這個 prompt: '{prompt}' 為基礎，以 array list 的方式，產生出額外 5 個相關的提問 prompt，我需要的 response 格式為 ['第一個產生的相似提問', '第二個產生的相似提問', '第三個產生的相似提問'] 這樣即可，不需要額外的其他內容，注意，相似提問的內容請以繁體中文呈現為主"
)
print(f"extend_prompts: {extend_prompts['response']}")

extend_prompts: ['觀察喜劇','喜劇演員','幽默寫作','搞笑影片','喜劇評論']


In [29]:

response = ollama.embeddings(
  prompt=prompt,
  model="llama3"
)

In [30]:
results = collection_query([response["embedding"]],6)
data = results['documents'][0]
# print(f"data: {data}")
print(results['metadatas'])
# print(results)

[[{'episode': 'EP228___海底抓取機器人_研發與實作的進程_博士的心境培養_ft__子源.txt', 'id': '1-55', 'name': 'techporn', 'type': 'podcast'}, {'episode': 'EP228___海底抓取機器人_研發與實作的進程_博士的心境培養_ft__子源.txt', 'id': '1-53', 'name': 'techporn', 'type': 'podcast'}, {'episode': 'EP228___海底抓取機器人_研發與實作的進程_博士的心境培養_ft__子源.txt', 'id': '1-42', 'name': 'techporn', 'type': 'podcast'}, {'episode': 'EP228___海底抓取機器人_研發與實作的進程_博士的心境培養_ft__子源.txt', 'id': '1-51', 'name': 'techporn', 'type': 'podcast'}, {'episode': 'EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt', 'id': '0-41', 'name': 'techporn', 'type': 'podcast'}, {'episode': 'EP225___哥倫比亞研究生_紐約生活與衝擊_創新開發的學習_ft__Tony.txt', 'id': '4-72', 'name': 'techporn', 'type': 'podcast'}]]


In [25]:
# print(collection.peek(10)['documents'])
print(collection.peek(100)['ids'])
print(collection.peek(100)['documents'])

[]
[]


In [95]:
ollama.pull(model="llama3")
output = ollama.generate(
    model="llama3",
    prompt=f"以繁體中文回答\n使用這筆資料: {data}, 來對這個提問\n{prompt}\n產生對應的回應"
)
print(output['response'])

😊

根據提供的資料，以下是兩隻動物之間的相似度排序：

1. 紅大袋鼠 (Pseudocheirus peregrinus) - 90%
2. 台灣長鬃山羊 (Muntiacus reevesi) - 80%

這兩個動物都是哺乳綱（Mammalia）下的偶蹄目（Artiodactyla）的成員，並且都有袋鼠科（Macropodidae）或牛科（Bovidae）等的關係。

在這兩隻動物中，紅大袋鼠是最相近於白狗（Canis lupus familiaris）的，因為：

* 都是哺乳綱下的偶蹄目。
* 都有袋鼠科或牛科的關係。
* 都有四肢和尾巴。

因此，根據資料庫中的資料，我們可以認為紅大袋鼠與白狗之間的相似度最高。 👍


In [70]:
delete_collection("docs")